# Testing Multitaper Spectrogram Computation Methods
Michael Nolan
2020.06.26

In [ ]:
from aopy import datareader, datafilter
import tfspec

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch

import fooof

In [ ]:
# load an example file
ecog_example_filepath = "/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/180325/001/rec001.LM1_ECOG_3.clfp.dat"
data, experiment, mask = datareader.load_ecog_clfp_data(ecog_example_filepath,t_range=(0,20))
data = data - data.mean(axis=0)

In [ ]:
num_ch,N = data.shape
srate = experiment['srate']
T = N/srate
print('Channels: {}\nSampling Rate: {}Hz\nSamples: {}\nTime: {}s'.format(num_ch,srate,N,T))

In [ ]:
T_plot = 60
ch_idx = 0
N_plot = data.shape[-1]
time_plot = np.arange(N_plot)/srate
f_inspect,ax = plt.subplots(2,1,figsize=(12,6),dpi=100,sharex=True)
ax[0].plot(time_plot,data[ch_idx,:N_plot])
ax[1].plot(time_plot,mask['hf'][:N_plot])
ax[1].plot(time_plot,mask['sat'][:N_plot])
ax[0].set_ylabel('ch. {} ($\mu V$)'.format(ch_idx))
ax[0].set_title('ECoG Data')
ax[1].set_ylabel('0/1')
ax[1].set_xlabel('time (s)')

In [ ]:
# data[:,mask['hf'] | mask['sat']] = 0.

In [ ]:
# spectrogram estimation
tapers = [0.3,5];
dn = 0.1;
T = 10
spec, f, ti, err = tfspec.tfspec(data[:,:T*srate], tapers=tapers, sampling=srate, dn=dn,fk=(0,120), contflag=True);

In [ ]:
spec[-1,-1,:10]

In [ ]:
# # filter bad regions by spectrogram power
# total_power_masked = 10*np.ma.log10(spec).mean(axis=-1)[0,:]
# mean_power = total_power_masked.mean()
# std_power = total_power_masked.std()
# time_mask = (total_power_masked.data < mean_power - std_power) | (total_power_masked.data > mean_power + std_power) | (total_power_masked.mask)

In [ ]:
# time_mask.mean()

In [ ]:
# plt.plot(f,10*np.log10(spec[0,~time_mask,:].mean(axis=0)/srate**2))
plt.plot(f,10*np.log10(spec[:,:45,:].mean(axis=(0,1))))

In [ ]:
_,ax = plt.subplots(1,1,figsize=(12,6))
# ax.imshow(10*np.log10(spec[0,np.logical_not(time_mask),:]).T,origin='bottom',clim=(-30,0),aspect='auto')
ax.imshow(10*np.log10(spec.mean(axis=0)).T,origin='bottom',clim=(30,50),aspect='auto')

In [ ]:
plt.